In [92]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
import random
from statsmodels.stats.proportion import binom_test
from statsmodels.stats.multitest import multipletests


In [2]:
def btest(pa1, pa2, seed=0, return_proportions=False):
    """ Performs genome wide binomial test between two groups of taxa
    Parameters
    ----------
    df1 : pd.DataFrame
        Rows are taxa, columns are genes
    df2 : pd.DataFrame
        Rows are taxa, columns are genes
    Returns
    -------
    pd.Series : list of genes associated with df1
    pd.Series : list of genes associated with df2
    """
    np.random.seed(seed)
    random.seed(seed)
    #pa1 = df1 > 0
    #pa2 = df2 > 0
    idx = list(set(pa1.columns) | set(pa2.columns))
    idx.sort()
    pa1 = pa1.sum(axis=0).reindex(idx).fillna(0)
    pa2 = pa2.sum(axis=0).reindex(idx).fillna(0)
    n = pa1 + pa2
    obs = list(zip(list(pa1.values), list((pa2.values + 1) / (pa2 + 1).sum()), list(n.values)))
    pvals = pd.Series([binom_test(a, n, b, 'two-sided') for (a, b, n) in obs],
                      index=n.index)
    if return_proportions:
        res = pd.DataFrame({'groupA': pa1, 'groupB': pa2, 'pval': pvals})
        def relabel_f(x):
            if x['groupA'] < x['groupB']:
                return 'groupB'
            else:
                return 'groupA'
        res['side'] = res.apply(relabel_f, axis=1)
        return res

    return pvals

In [3]:
def parse_genome(df):
    genome_id = df['#query'][0].split('_')[0]
    keggs = df['KEGG_ko'].replace('-', None).dropna()
    keggs = list(map(lambda x: x.split(','), keggs.values))
    keggs = sum(keggs, [])
    keggs = pd.DataFrame({'KEGG_ko': keggs})
    keggs['genome_id'] = genome_id
    return keggs


In [211]:
def get_kegg_from_annotation(ids):
    """
    ids: Species_rep of the top/bottom k microbes
    """
    df_list = []
    for i in ids:
        f_name = "../../permutation_table/AD_species/{}_eggNOG.tsv".format(i)
        df_parsed = parse_genome(pd.read_table(f_name))
        df_list.append(df_parsed)
    df_cat = pd.concat(df_list, axis = 0)
    kegg_counts = to_sparse_matrix(df_cat)
    return kegg_counts
 

In [84]:
def to_sparse_matrix(func_df, genome_id='genome_id', kegg_id='KEGG_ko'):
    # create genome-specific index
    ogus = list(set(func_df[genome_id]))
    ogu_lookup = pd.Series(np.arange(0, len(ogus)), ogus)
    # create KEGG-specific index
    keggs = list(set(func_df[kegg_id]))
    kegg_lookup = pd.Series(np.arange(0, len(keggs)), keggs)
    # rename names as numbers
    ogu_id = func_df[genome_id].apply(lambda x: ogu_lookup.loc[x]).astype(np.int64)
    kegg_id = func_df[kegg_id].apply(lambda x: kegg_lookup.loc[x]).astype(np.int64)
    # assign the presence / absence of a gene
    func_df['count'] = 1
    c = func_df['count'].values
    # format into a matrix
    data = coo_matrix((c, (ogu_id, kegg_id)))
    ko_ogu = pd.DataFrame(data.todense(), index=ogus, columns=keggs)
    return ko_ogu

In [210]:
table_AD = pd.read_table("../../permutation_table/Laske2022AD_new.txt", index_col = 0) 

In [51]:
#select 10 microbes as a test table
table_test_ad = table_AD.head(10)

In [52]:
#filter metadata
Metadata = pd.read_table("../table/metadata_12dataset_1258_adonis.txt", index_col = 0)

In [53]:
Metadata_AD = Metadata[(Metadata['Study'] == "Laske2022AD")]

In [55]:
#Metadata_AD
table_test_ad_T = table_test_ad.T

In [60]:
table_test_ad_T.reindex(Metadata_AD.index)
table_test_ad_T

species,1XD42-69 sp014287635,28L sp000177555,43-108 sp001915545,51-20 sp001917175,51-20 sp900539605,51-20 sp900542055,51-20 sp900762565,AF33-28 sp003477885,AM07-15 sp003477405,AM51-8 sp003478275
ERR9578158,4848,0,233,50,13,83,197,1771,2170,4716
ERR9578316,22084,0,242,2448,44,463,274,1865,1096,561
ERR9578225,1664,11,969,0,0,0,210,1655,5429,1850
ERR9578267,1379,0,533,17,0,22,0,1465,1436,538
ERR9578253,446,0,470,0,0,0,0,1894,4302,633
...,...,...,...,...,...,...,...,...,...,...
ERR9578173,1139,0,37,0,0,0,0,3744,4292,1222
ERR9578300,908,0,262,64,22,298,11,2765,2222,18981
ERR9578239,537,0,650,270,364,56,59,4563,2500,379
ERR9578296,364,0,171,0,0,0,0,514,1129,640


In [203]:
table_test_ad_T.to_csv("../../permutation_table/table_AD_test.txt", sep = "\t")
Metadata_AD.to_csv("../../permutation_table/metadata_AD_test.txt", sep = "\t")

In [221]:
pd.read_table("/Users/dongminjin/Documents/github_rep/permutation_table/eggNOG_species_rep.txt", sep = '\t')

,Species,Species_rep
0,GCA-900066495 sp902362365,MGYG000000001
1,Blautia_A faecis,MGYG000000002
2,Alistipes shahii,MGYG000000003
3,Anaerotruncus colihominis,MGYG000000004
4,Terrisporobacter glycolicus_A,MGYG000000005
...,...,...
289227,AM51-8 sp003478275,MGYG000000280
289228,Phascolarctobacterium sp000436095,MGYG000004337
289229,CAG-488 sp000434055,MGYG000000669
289230,Butyrivibrio_A crossotus,MGYG000001319


In [212]:
def _test(X, y, k, p, M):
    """
    X: count matrix of microbes (m microbes X n samples)
    y: disease lables for samples (n samples)
    G: gene table
    k: top k microbes, eg k = 100
    p: number of permutations, eg p = 1000
    M: metadata = pd.read_table('../table/eggNOG_species_rep.txt')
    """
    # Convert counts and group labels to PyDESeq2 input format
    dds = DeseqDataSet(
        X, 
        y, 
        design_factors = "disease", # compare samples based on the "disease"
        refit_cooks = True,
        n_cpus = 8
    )
    dds.deseq2()
    stat_res = DeseqStats(dds, n_cpus=8)
    res = stat_res.summary()
    res_df = stat_res.results_df
    res_df["CI_95"] = res_df["log2FoldChange"] + res_df['lfcSE'] * 1.96
    res_df["CI_5" ] = res_df["log2FoldChange"] - res_df['lfcSE'] * 1.96
    #top k microbes: the ones with largest CI_5
    top = res_df.sort_values(by=['CI_5'],ascending=False).head(k)
    #bottom k microbes: the ones with smallest CI_95
    bot = res_df.sort_values(by=['CI_95'],ascending=True).head(k)
    #convert microbe names to species ids
    top_microbe = set(top.index)
    bot_microbe = set(bot.index)
    top_rep = M[M['Species'].isin(top_microbe)]
    bot_rep = M[M['Species'].isin(bot_microbe)]
    top_id = top_rep['Species_rep'].drop_duplicates()
    bot_id = bot_rep['Species_rep'].drop_duplicates()
    #wget all ids: dir: /mnt/home/djin/ceph/Permutation_test/table/Species
    #get G gene matrix
    top_gene_matrix = get_kegg_from_annotation(top_id)
    bot_gene_matrix = get_kegg_from_annotation(bot_id)

    #binomial test Return number of genes elevated in top
    """
    C = btest(G[top], G[bot])
    return C
    """
    kegg = btest(top_gene_matrix,bot_gene_matrix,return_proportions=True)
    kegg_top = kegg.loc[kegg['side'] == 'groupA']
    kegg_top_sig = kegg_top.loc[kegg_top['pval'] <= 0.001]
    C = len(kegg_top_sig.index)
    return C

In [213]:
T = _test(table_test_ad_T, Metadata_AD, k=2, p=10, M=M)

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,1667.080704,-0.557058,0.213314,-2.611447,9.016003e-03,0.045080
28L sp000177555,1.770149,-0.364849,1.419733,-0.256985,7.971907e-01,0.797191
43-108 sp001915545,710.528771,-0.538207,0.299114,-1.799338,7.196526e-02,0.239884
51-20 sp001917175,6084.375509,3.133779,0.620165,5.053137,4.346110e-07,0.000004
51-20 sp900539605,72.803533,0.584788,0.892070,0.655540,5.121201e-01,0.797191
51-20 sp900542055,153.775373,0.174070,0.562610,0.309398,7.570187e-01,0.797191
51-20 sp900762565,184.591470,-0.390259,0.591040,-0.660291,5.090668e-01,0.797191
AF33-28 sp003477885,1773.295421,0.143777,0.158251,0.908537,3.635947e-01,0.797191
AM07-15 sp003477405,2885.886582,0.077575,0.180191,0.430513,6.668227e-01,0.797191


In [214]:
T

1420

In [198]:
def permutation_test(X, y, k, p, M):
    T = _test(X, y, k, p, M)
    for i in range(p):
        #shuffle the group lables
        y_permutated = np.random.permutation(y['disease'])
        y_permutated = pd.DataFrame(y_permutated, index=y.index)
        y_permutated.columns = ['disease']
        y_permutated.reindex(counts.index)
        T_ = _test(X, y_permutated, k, p, M)
    return T, (T > T_) / (p)
    #return T, (T > T_) / (p+1)

In [217]:
a, b = permutation_test(table_test_ad_T,Metadata_AD, k=2, p=100, M= M )

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,1667.080704,-0.557058,0.213314,-2.611447,9.016003e-03,0.045080
28L sp000177555,1.770149,-0.364849,1.419733,-0.256985,7.971907e-01,0.797191
43-108 sp001915545,710.528771,-0.538207,0.299114,-1.799338,7.196526e-02,0.239884
51-20 sp001917175,6084.375509,3.133779,0.620165,5.053137,4.346110e-07,0.000004
51-20 sp900539605,72.803533,0.584788,0.892070,0.655540,5.121201e-01,0.797191
51-20 sp900542055,153.775373,0.174070,0.562610,0.309398,7.570187e-01,0.797191
51-20 sp900762565,184.591470,-0.390259,0.591040,-0.660291,5.090668e-01,0.797191
AF33-28 sp003477885,1773.295421,0.143777,0.158251,0.908537,3.635947e-01,0.797191
AM07-15 sp003477405,2885.886582,0.077575,0.180191,0.430513,6.668227e-01,0.797191


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,1667.080704,0.145766,0.216908,0.672018,0.501572,0.835954
28L sp000177555,1.770149,0.210734,1.418213,0.148591,0.881876,0.923612
43-108 sp001915545,622.324965,-0.114746,0.286296,-0.400794,0.688572,0.923612
51-20 sp001917175,3222.692822,-0.975184,0.613552,-1.589407,0.111969,0.457338
51-20 sp900539605,72.803533,-1.164162,0.885972,-1.313994,0.188848,0.457338
51-20 sp900542055,153.775373,-0.053961,0.562767,-0.095885,0.923612,0.923612
51-20 sp900762565,184.591470,0.821120,0.587997,1.396469,0.162573,0.457338
AF33-28 sp003477885,1773.295421,-0.288167,0.157060,-1.834762,0.066541,0.457338
AM07-15 sp003477405,2885.886582,-0.018358,0.180267,-0.101839,0.918884,0.923612


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


...

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,1667.080704,0.145766,0.216908,0.672018,0.501572,0.835954
28L sp000177555,1.770149,0.210734,1.418213,0.148591,0.881876,0.923612
43-108 sp001915545,622.324965,-0.114746,0.286296,-0.400794,0.688572,0.923612
51-20 sp001917175,3222.692822,-0.975184,0.613552,-1.589407,0.111969,0.457338
51-20 sp900539605,72.803533,-1.164162,0.885972,-1.313994,0.188848,0.457338
51-20 sp900542055,153.775373,-0.053961,0.562767,-0.095885,0.923612,0.923612
51-20 sp900762565,184.591470,0.821120,0.587997,1.396469,0.162573,0.457338
AF33-28 sp003477885,1773.295421,-0.288167,0.157060,-1.834762,0.066541,0.457338
AM07-15 sp003477405,2885.886582,-0.018358,0.180267,-0.101839,0.918884,0.923612


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,1667.080704,0.145766,0.216908,0.672018,0.501572,0.835954
28L sp000177555,1.770149,0.210734,1.418213,0.148591,0.881876,0.923612
43-108 sp001915545,622.324965,-0.114746,0.286296,-0.400794,0.688572,0.923612
51-20 sp001917175,3222.692822,-0.975184,0.613552,-1.589407,0.111969,0.457338
51-20 sp900539605,72.803533,-1.164162,0.885972,-1.313994,0.188848,0.457338
51-20 sp900542055,153.775373,-0.053961,0.562767,-0.095885,0.923612,0.923612
51-20 sp900762565,184.591470,0.821120,0.587997,1.396469,0.162573,0.457338
AF33-28 sp003477885,1773.295421,-0.288167,0.157060,-1.834762,0.066541,0.457338
AM07-15 sp003477405,2885.886582,-0.018358,0.180267,-0.101839,0.918884,0.923612


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,1667.080704,0.145766,0.216908,0.672018,0.501572,0.835954
28L sp000177555,1.770149,0.210734,1.418213,0.148591,0.881876,0.923612
43-108 sp001915545,622.324965,-0.114746,0.286296,-0.400794,0.688572,0.923612
51-20 sp001917175,3222.692822,-0.975184,0.613552,-1.589407,0.111969,0.457338
51-20 sp900539605,72.803533,-1.164162,0.885972,-1.313994,0.188848,0.457338
51-20 sp900542055,153.775373,-0.053961,0.562767,-0.095885,0.923612,0.923612
51-20 sp900762565,184.591470,0.821120,0.587997,1.396469,0.162573,0.457338
AF33-28 sp003477885,1773.295421,-0.288167,0.157060,-1.834762,0.066541,0.457338
AM07-15 sp003477405,2885.886582,-0.018358,0.180267,-0.101839,0.918884,0.923612


In [218]:
a, b

(1420, 0.01)

In [192]:
y = Metadata_AD
counts = table_test_ad_T
y_permutated = np.random.permutation(y['disease'])
y_permutated = pd.DataFrame(y_permutated, index=y.index)
y_permutated.columns = ['disease']
y_permutated.reindex(counts.index)
y_permutated

,disease
featureid,
ERR9578157,Healthy
ERR9578158,AD
ERR9578159,Healthy
ERR9578160,AD
ERR9578161,Healthy
...,...
ERR9578327,AD
ERR9578328,AD
ERR9578329,Healthy


In [61]:
dds = DeseqDataSet(table_test_ad_T, Metadata_AD, design_factors = "disease", refit_cooks = True, n_cpus = 8)
dds.deseq2()
stat_res = DeseqStats(dds, n_cpus=8)
stat_res.summary()

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,1667.080704,-0.557058,0.213314,-2.611447,9.016003e-03,0.045080
28L sp000177555,1.770149,-0.364849,1.419733,-0.256985,7.971907e-01,0.797191
43-108 sp001915545,710.528771,-0.538207,0.299114,-1.799338,7.196526e-02,0.239884
51-20 sp001917175,6084.375509,3.133779,0.620165,5.053137,4.346110e-07,0.000004
51-20 sp900539605,72.803533,0.584788,0.892070,0.655540,5.121201e-01,0.797191
51-20 sp900542055,153.775373,0.174070,0.562610,0.309398,7.570187e-01,0.797191
51-20 sp900762565,184.591470,-0.390259,0.591040,-0.660291,5.090668e-01,0.797191
AF33-28 sp003477885,1773.295421,0.143777,0.158251,0.908537,3.635947e-01,0.797191
AM07-15 sp003477405,2885.886582,0.077575,0.180191,0.430513,6.668227e-01,0.797191


In [62]:
M = pd.read_table('../table/eggNOG_species_rep.txt')

In [68]:
k = 100
top

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,CI_95,CI_5
species,,,,,,,,
51-20 sp001917175,6084.375509,3.133779,0.620165,5.053137,4.346110e-07,0.000004,4.349303,1.918256
AF33-28 sp003477885,1773.295421,0.143777,0.158251,0.908537,3.635947e-01,0.797191,0.453950,-0.166395
AM07-15 sp003477405,2885.886582,0.077575,0.180191,0.430513,6.668227e-01,0.797191,0.430750,-0.275600
AM51-8 sp003478275,2345.324981,0.127499,0.235829,0.540641,5.887550e-01,0.797191,0.589724,-0.334726
51-20 sp900542055,153.775373,0.174070,0.562610,0.309398,7.570187e-01,0.797191,1.276786,-0.928645
1XD42-69 sp014287635,1667.080704,-0.557058,0.213314,-2.611447,9.016003e-03,0.045080,-0.138963,-0.975153
43-108 sp001915545,710.528771,-0.538207,0.299114,-1.799338,7.196526e-02,0.239884,0.048056,-1.124470
51-20 sp900539605,72.803533,0.584788,0.892070,0.655540,5.121201e-01,0.797191,2.333245,-1.163670
51-20 sp900762565,184.591470,-0.390259,0.591040,-0.660291,5.090668e-01,0.797191,0.768180,-1.548697


In [112]:
k = 2

In [113]:
res = stat_res.summary()
res_df = stat_res.results_df
res_df["CI_95"] = res_df["log2FoldChange"] + res_df['lfcSE'] * 1.96
res_df["CI_5" ] = res_df["log2FoldChange"] - res_df['lfcSE'] * 1.96
#top k microbes: the ones with largest CI_5
top = res_df.sort_values(by=['CI_5'],ascending=False).head(k)
#bottom k microbes: the ones with smallest CI_95
bot = res_df.sort_values(by=['CI_95'],ascending=True).head(k)
#convert microbe names to species ids
top_microbe = set(top.index)
bot_microbe = set(bot.index)
top_rep = M[M['Species'].isin(top_microbe)]
bot_rep = M[M['Species'].isin(bot_microbe)]
top_id = top_rep['Species_rep'].drop_duplicates()
bot_id = bot_rep['Species_rep'].drop_duplicates()

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,1667.080704,-0.557058,0.213314,-2.611447,9.016003e-03,0.045080
28L sp000177555,1.770149,-0.364849,1.419733,-0.256985,7.971907e-01,0.797191
43-108 sp001915545,710.528771,-0.538207,0.299114,-1.799338,7.196526e-02,0.239884
51-20 sp001917175,6084.375509,3.133779,0.620165,5.053137,4.346110e-07,0.000004
51-20 sp900539605,72.803533,0.584788,0.892070,0.655540,5.121201e-01,0.797191
51-20 sp900542055,153.775373,0.174070,0.562610,0.309398,7.570187e-01,0.797191
51-20 sp900762565,184.591470,-0.390259,0.591040,-0.660291,5.090668e-01,0.797191
AF33-28 sp003477885,1773.295421,0.143777,0.158251,0.908537,3.635947e-01,0.797191
AM07-15 sp003477405,2885.886582,0.077575,0.180191,0.430513,6.668227e-01,0.797191


In [114]:
top_microbe

{'51-20 sp001917175', 'AF33-28 sp003477885'}

In [115]:
bot_microbe

{'1XD42-69 sp014287635', '43-108 sp001915545'}

In [116]:
#get G gene matrix
top_gene_matrix = get_kegg_from_annotation(top_id)
bot_gene_matrix = get_kegg_from_annotation(bot_id)


In [120]:
kegg = btest(top_gene_matrix,bot_gene_matrix,return_proportions=True)
kegg_top = kegg.loc[kegg['side'] == 'groupA']
kegg_top_sig = kegg_top.loc[kegg_top['pval'] <= 0.001]

In [126]:
len(kegg_top_sig.index)

1420

In [129]:
table_AD = pd.read_table("../table/Laske2022AD_new.txt", index_col = 0)
table_UC = pd.read_table("../table/Nielsen2014UC_new.txt", index_col = 0)
table_CD = pd.read_table("../table/Franzosa2019IBD_CD_new.txt", index_col = 0)

In [136]:
set1 = {1, 2, 3}
set2 = {3, 4, 5}
set3 = {5, 6, 7}

combined_set = set1 | set2 | set3

print(combined_set)


{1, 2, 3, 4, 5, 6, 7}


In [138]:
set_AD = set(table_AD.index)
set_CD = set(table_CD.index)
set_UC = set(table_UC.index)

In [139]:
combined_microbes = set_AD | set_CD | set_UC

In [143]:
combined_microbes  = pd.DataFrame(combined_microbes) 

In [146]:
combined_microbes.to_csv('../table/microbes.csv') 

In [151]:
microbes = pd.read_csv('../table/microbes.csv', index_col = 0)
ids = set(microbes['0'])

In [153]:
ids_table = M[M['Species'].isin(ids)]

In [155]:
annotated_ids = ids_table['Species_rep'].drop_duplicates()

In [157]:
annotated_ids = pd.DataFrame(annotated_ids) 
annotated_ids.to_csv('../table/annotated_ids.csv') 

In [165]:
ids = pd.read_csv('../table/annotated_ids.csv', index_col = 0)
ids = set(ids['Species_rep'])

#/mnt/home/djin/ceph/Permutation_test/table/Species